In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

from xxchallenge import *


# Parameters
A = tlm.parameter(-1.62)
T = 100.0
NY = tlm.parameter(90.0)
NZ = tlm.parameter(0.0)
RZ = 4.34

cylinder = tlm.ImplicitCylinder(*torch.tensor([-50/2, 50/2, 37.02/2], dtype=torch.float64).unbind())
rod_data = StoreVar(lambda data: data)

# Primary mirror
primary = tlm.Parabola(1800, A=A, normalize=True)
primary_data = StoreVar(lambda data: data)

# Optical model
optics = tlm.Sequential(
    tlm.Gap(-1000),
    XXLightSource.load(),
    #RaysViewerPlane(2000, "input"),
    tlm.AbsolutePosition(x=-150),
    tlm.Translate3D(y=T),
    tlm.Rotate3D(z=RZ),
    primary_data,
    tlm.ReflectiveSurface(primary),
    Focus(primary),
    #tlm.FocalPoint(),
    BoxLoss(450, 1.0, 0.1),
    tlm.Rotate3D(y=NY, z=NZ),
    rod_data,
    NonImagingRod(cylinder),
)

xxrender(optics, sampling={"xx": 500, "letter": "negative"})

In [ ]:
from itertools import product

sampling = {"xx": 30000, "letter": "positive"}

#gridsearch1d(optics, NY, torch.tensor([-90, 0, 90]), sampling)

parameters = [NY, NZ]
spaces = [
    torch.tensor([0, 90]),
    torch.tensor([0, 90]),
]

# gridsearchnd(optics, [NY, NZ], spaces, sampling)

#gridsearch1d(optics, NZ, torch.tensor([-90, -45, 0, 45, 90]), sampling)


In [ ]:
param_groups = [
    #{'params': [RY], 'lr': 1e-4},
    #{'params': [RZ], 'lr': 1e-6},
    {'params': [A], 'lr': 1e-4},
]

record = tlm.optimize(
    optics,
    optimizer = optim.SGD(param_groups),
    sampling = {"xx": 50000, "disable_viewer": True, "letter": "positive"},
    dim = 3,
    num_iter = 10
)


plot_record(record, param_groups, optics)
record.best()

In [ ]:
print("A", A)
F = 1./(primary._sag.unnorm(primary.diameter / 2) * 4)
print("F", F.item())
print("RZ", RZ)

xxrender(optics, end=50, sampling={"xx": 500, "disable_viewer": True, "letter": "both"})

In [ ]:
with torch.no_grad():
    part_primary = tess_mirror(xxgrid(499, 100), primary_data.value.tf(), primary)
    #part_secondary = tess_mirror(xxgrid(1.1*secondary.diameter / 2, 100), secondary_data.value.tf(), secondary)

part_sides = makesides(part_primary.vectors.dtype)  

final_part = mesh.Mesh(np.concatenate([
    part_primary.data,
    #part_secondary.data,
    part_sides.data
]))

final_part.save("parabola2.stl")

# Print rod position
target = rod_data.value.target()
print("ROD X", target[1].item())
print("ROD Y", target[2].item())
print("ROD Z", -target[0].item())